In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya_speech.train.model.conformer as conformer
import malaya_speech.train.model.transducer as transducer
import malaya_speech
import tensorflow as tf
import numpy as np
import json
from glob import glob
from sklearn.utils import shuffle
from pydub import AudioSegment

In [3]:
subwords = malaya_speech.subword.load('transducer.subword')
vocabs = {i: subwords._id_to_subword(i) for i in range(subwords.vocab_size - 1)}
vocabs[-1] = ''

In [4]:
featurizer = malaya_speech.tf_featurization.STTFeaturizer(
    normalize_per_feature = True
)

In [5]:
n_mels = 80
sr = 16000
maxlen = 18
minlen_text = 1

def mp3_to_wav(file, sr = sr):
    audio = AudioSegment.from_file(file)
    audio = audio.set_frame_rate(sr).set_channels(1)
    sample = np.array(audio.get_array_of_samples())
    return malaya_speech.astype.int_to_float(sample), sr


def generate(file):
    with open(file) as fopen:
        dataset = json.load(fopen)
    audios, cleaned_texts = dataset['X'], dataset['Y']
    for i in range(len(audios)):
        try:
            if audios[i].endswith('.mp3'):
                # print('found mp3', audios[i])
                wav_data, _ = mp3_to_wav(audios[i])
            else:
                wav_data, _ = malaya_speech.load(audios[i], sr = sr)

            if (len(wav_data) / sr) > maxlen:
                # print(f'skipped audio too long {audios[i]}')
                continue

            if len(cleaned_texts[i]) < minlen_text:
                # print(f'skipped text too short {audios[i]}')
                continue

            t = malaya_speech.subword.encode(
                subwords, cleaned_texts[i], add_blank = False
            )
            back = np.zeros(shape=(2000,))
            front = np.zeros(shape=(200,))
            wav_data = np.concatenate([front, wav_data, back], axis=-1)

            yield {
                'waveforms': wav_data,
                'targets': t,
                'targets_length': [len(t)],
            }
        except Exception as e:
            print(e)


def preprocess_inputs(example):
    s = featurizer.vectorize(example['waveforms'])
    mel_fbanks = tf.reshape(s, (-1, n_mels))
    length = tf.cast(tf.shape(mel_fbanks)[0], tf.int32)
    length = tf.expand_dims(length, 0)
    example['inputs'] = mel_fbanks
    example['inputs_length'] = length
    example.pop('waveforms', None)
    return example


def get_dataset(
    file,
    batch_size = 3,
    shuffle_size = 20,
    thread_count = 24,
    maxlen_feature = 1800,
):
    def get():
        dataset = tf.data.Dataset.from_generator(
            generate,
            {
                'waveforms': tf.float32,
                'targets': tf.int32,
                'targets_length': tf.int32,
            },
            output_shapes = {
                'waveforms': tf.TensorShape([None]),
                'targets': tf.TensorShape([None]),
                'targets_length': tf.TensorShape([None]),
            },
            args = (file,),
        )
        dataset = dataset.prefetch(tf.contrib.data.AUTOTUNE)
        dataset = dataset.map(
            preprocess_inputs, num_parallel_calls = thread_count
        )
        dataset = dataset.padded_batch(
            batch_size,
            padded_shapes = {
                'inputs': tf.TensorShape([None, n_mels]),
                'inputs_length': tf.TensorShape([None]),
                'targets': tf.TensorShape([None]),
                'targets_length': tf.TensorShape([None]),
            },
            padding_values = {
                'inputs': tf.constant(0, dtype = tf.float32),
                'inputs_length': tf.constant(0, dtype = tf.int32),
                'targets': tf.constant(0, dtype = tf.int32),
                'targets_length': tf.constant(0, dtype = tf.int32),
            },
        )
        return dataset

    return get

In [6]:
dev_dataset = get_dataset('bahasa-asr-test.json')()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [7]:
features = dev_dataset.make_one_shot_iterator().get_next()
features

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


{'targets': <tf.Tensor 'IteratorGetNext:2' shape=(?, ?) dtype=int32>,
 'targets_length': <tf.Tensor 'IteratorGetNext:3' shape=(?, ?) dtype=int32>,
 'inputs': <tf.Tensor 'IteratorGetNext:0' shape=(?, ?, 80) dtype=float32>,
 'inputs_length': <tf.Tensor 'IteratorGetNext:1' shape=(?, ?) dtype=int32>}

In [8]:
training = True

In [9]:
config = malaya_speech.config.conformer_tiny_encoder_config
config['dropout'] = 0.0
conformer_model = conformer.Model(
    kernel_regularizer = None, bias_regularizer = None, **config
)
decoder_config = malaya_speech.config.conformer_tiny_decoder_config
decoder_config['embed_dropout'] = 0.0
transducer_model = transducer.rnn.Model(
    conformer_model, vocabulary_size = subwords.vocab_size, **decoder_config
)
targets_length = features['targets_length'][:, 0]
v = tf.expand_dims(features['inputs'], -1)
z = tf.zeros((tf.shape(features['targets'])[0], 1), dtype = tf.int32)
c = tf.concat([z, features['targets']], axis = 1)

logits = transducer_model([v, c, targets_length + 1], training = training)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
decoded = transducer_model.greedy_decoder(v, features['inputs_length'][:, 0], training = training)
decoded

(<tf.Tensor 'while/Exit_1:0' shape=(?, ?) dtype=int32>,
 <tf.Tensor 'while/Exit_2:0' shape=(?, ?, 1030) dtype=float32>)

In [11]:
decoded = decoded[0]

In [12]:
encoded = transducer_model.encoder(v, training = training)
encoded = tf.identity(encoded, name = 'encoded')
encoded_placeholder = tf.placeholder(tf.float32, [config['dmodel']], name = 'encoded_placeholder')
predicted_placeholder = tf.placeholder(tf.int32, None, name = 'predicted_placeholder')
t = transducer_model.predict_net.get_initial_state().shape
states_placeholder = tf.placeholder(tf.float32, [int(i) for i in t], name = 'states_placeholder')

ytu, new_states = transducer_model.decoder_inference(
    encoded=encoded_placeholder,
    predicted=predicted_placeholder,
    states=states_placeholder,
    training = training,
    use_softmax = True
)

ytu = tf.identity(ytu, name = 'ytu')
new_states = tf.identity(new_states, name = 'new_states')
ytu, new_states

(<tf.Tensor 'ytu:0' shape=(1030,) dtype=float32>,
 <tf.Tensor 'new_states:0' shape=(1, 2, 1, 128) dtype=float32>)

In [13]:
initial_states = transducer_model.predict_net.get_initial_state()
initial_states = tf.identity(initial_states, name = 'initial_states')

In [14]:
from pyctcdecode.language_model import LanguageModel
import kenlm

kenlm_model = kenlm.Model('model.trie.klm')
language_model = LanguageModel(kenlm_model, alpha = 0.05, beta = 1.0)

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [16]:
saver = tf.train.Saver(var_list = tf.trainable_variables())
saver.restore(sess, 'asr-tiny-conformer-transducer-v3/model.ckpt-575000')

INFO:tensorflow:Restoring parameters from asr-tiny-conformer-transducer-v3/model.ckpt-575000


In [17]:
import collections
import numpy as np
import tensorflow as tf
import re
import heapq
from typing import Any, Collection, Dict, Iterable, List, Optional, Tuple, Union
np.set_printoptions(threshold=3)

LMState = Optional[Union[kenlm.State, List[kenlm.State]]]

BeamHypothesis_LM = collections.namedtuple(
    'BeamHypothesis_LM', ('score', 'score_lm', 'prediction', 'states', 'text', 'next_word', 'word_part')
)

def apply_temp(logits_BxN, temperature):
    uniform_noise_BxN = np.random.uniform(size = logits_BxN.shape)
    logits_BxN += -np.log(-np.log(uniform_noise_BxN)) * temperature
    return logits_BxN

def _sort_and_trim_beams(beams: list, beam_width: int) -> list:
    return heapq.nlargest(beam_width, beams, key=lambda x: x.score_lm)

def prune_space(string):
    return re.sub(r'[ ]+', ' ', string).strip()

def _merge_tokens(token_1, token_2) -> str:
    if len(token_2) == 0:
        text = token_1
    elif len(token_1) == 0:
        text = token_2
    else:
        text = token_1 + " " + token_2
    return prune_space(text)

def _prune_history(beams, lm_order: int):
    """Filter out beams that are the same over max_ngram history.
    Since n-gram language models have a finite history when scoring a new token, we can use that
    fact to prune beams that only differ early on (more than n tokens in the past) and keep only the
    higher scoring ones. Note that this helps speed up the decoding process but comes at the cost of
    some amount of beam diversity. If more than the top beam is used in the output it should
    potentially be disabled.
    """
    # let's keep at least 1 word of history
    min_n_history = max(1, lm_order - 1)
    seen_hashes = set()
    filtered_beams = []
    # for each beam after this, check if we need to add it
    for beam in beams:
        text = beam.text
        next_word = beam.next_word
        word_part = beam.word_part
        last_char = beam.prediction[-1]
        logit_score = beam.score
        hash_idx = (tuple(text.split()[-min_n_history:]), word_part, last_char)
        if hash_idx not in seen_hashes:
            beam_hyp = BeamHypothesis_LM(
                score = logit_score,
                score_lm = 0.0,
                prediction = beam.prediction,
                states = beam.states,
                text = text,
                next_word = next_word,
                word_part = word_part
            )
            filtered_beams.append(beam_hyp)
            seen_hashes.add(hash_idx)
    return filtered_beams

def get_lm_beams(beams, cached_lm_scores,
        cached_partial_token_scores,
        is_eos: bool = False):
    
    new_beams = []
    for beam in beams:
        text = beam.text
        next_word = beam.next_word
        word_part = beam.word_part
        last_char = beam.prediction[-1]
        logit_score = beam.score
        new_text = _merge_tokens(text, next_word)
        if new_text not in cached_lm_scores:
            _, prev_raw_lm_score, start_state = cached_lm_scores[text]
            score, end_state = language_model.score(start_state, next_word, is_last_word=is_eos)
            raw_lm_score = prev_raw_lm_score + score
            lm_hw_score = raw_lm_score
            cached_lm_scores[new_text] = (lm_hw_score, raw_lm_score, end_state)
        lm_score, _, _ = cached_lm_scores[new_text]
        if len(word_part) > 0:
            if word_part not in cached_partial_token_scores:
                cached_partial_token_scores[word_part] = language_model.score_partial_token(
                    word_part
                )
            lm_score += cached_partial_token_scores[word_part]
        beam_hyp = BeamHypothesis_LM(
            score = logit_score,
            score_lm = logit_score + lm_score,
            prediction = beam.prediction,
            states = beam.states,
            text = new_text,
            next_word = '',
            word_part = word_part
        )
        new_beams.append(beam_hyp)

    return new_beams
            
        
def transducer_lm(
    enc,
    total,
    initial_states,
    encoded_placeholder,
    predicted_placeholder,
    states_placeholder,
    ytu,
    new_states,
    sess,
    beam_width = 10,
    token_min_logp = -20.0,
    beam_prune_logp = -5.0,
    temperature = 0.0,
    score_norm = True,
):
    kept_hyps = [
        BeamHypothesis_LM(score = 0.0, score_lm = 0.0,
                          prediction = [0], states = initial_states, 
                          text = '', next_word = '', word_part = '')
    ]
    start_state = kenlm.State()
    cached_lm_scores: Dict[str, Tuple[float, float, LMState]] = {
        '': (0.0, 0.0, language_model.get_start_state())
    }
    cached_p_lm_scores: Dict[str, float] = {}
    B = kept_hyps
    for i in range(total):
        A = B
        B = []
        while True:
            y_hat = max(A, key = lambda x: x.score)
            A.remove(y_hat)
            ytu_, new_states_ = sess.run(
                [ytu, new_states],
                feed_dict = {
                    encoded_placeholder: enc[i],
                    predicted_placeholder: y_hat.prediction[-1],
                    states_placeholder: y_hat.states,
                },
            )
            ytu_ = np.log(ytu_)
            if temperature > 0:
                ytu_ = apply_temp(ytu_, temperature=temperature)
            # ytu_ = np.clip(ytu_, np.log(1e-15), 0)
            B.append(BeamHypothesis_LM(
                score=y_hat.score + ytu_[0],
                score_lm = 0.0,
                prediction=y_hat.prediction,
                states=y_hat.states,
                text=y_hat.text,
                next_word=y_hat.next_word,
                word_part=y_hat.word_part,
            ))
            ytu_ = ytu_[1:]
            max_idx = ytu_.argmax()
            idx_list = set(np.where(ytu_ >= token_min_logp)[0]) | {max_idx}
            for k in idx_list:
                w = vocabs[k]
                if isinstance(w, bytes):
                    w = w.decode(encoding = 'ISO-8859-1')
                w = w.replace('_', ' ')
                s = y_hat.score + ytu_[k]
                p = y_hat.prediction + [k + 1]
            
                if w[-1] == ' ':
                    beam_hyp = BeamHypothesis_LM(
                        score = s,
                        score_lm = 0.0,
                        prediction = p,
                        states = new_states_,
                        text = y_hat.text,
                        next_word = y_hat.word_part + w,
                        word_part = ''
                    )
                else:
                    beam_hyp = BeamHypothesis_LM(
                        score = s,
                        score_lm = 0.0,
                        prediction = p,
                        states = new_states_,
                        text = y_hat.text,
                        next_word = y_hat.next_word,
                        word_part = y_hat.word_part + w
                    )
                A.append(beam_hyp)
            
            scored_beams = get_lm_beams(A, cached_lm_scores, cached_p_lm_scores)
            max_beam = max(scored_beams, key = lambda x: x.score_lm)
            max_score = max_beam.score_lm
            scored_beams = [b for b in scored_beams if b.score_lm >= max_score + beam_prune_logp]
            trimmed_beams = _sort_and_trim_beams(scored_beams, beam_width = beam_width)
            A = _prune_history(trimmed_beams, lm_order=language_model.order)
        
            max_A = max(A, key=lambda x: x.score)
            hyps_max = max_A.score
            kept_most_prob = [hyp for hyp in B if hyp.score > hyps_max]
            if len(kept_most_prob) >= beam_width:
                B = kept_most_prob
                break
            
    new_beams = []
    for beam in B:
        text = beam.text
        next_word = beam.next_word
        word_part = beam.word_part
        last_char = beam.prediction[-1]
        logit_score = beam.score
        beam_hyp = BeamHypothesis_LM(
            score = logit_score,
            score_lm = 0.0,
            prediction = beam.prediction,
            states = beam.states,
            text = text,
            next_word = word_part,
            word_part = ''
        )
        new_beams.append(beam_hyp)
    scored_beams = get_lm_beams(new_beams, cached_lm_scores, cached_p_lm_scores,is_eos=True)
    max_beam = max(scored_beams, key = lambda x: x.score_lm)
    max_score = max_beam.score_lm
    scored_beams = [b for b in scored_beams if b.score_lm >= max_score + beam_prune_logp]
    trimmed_beams = _sort_and_trim_beams(scored_beams, beam_width)

    if score_norm:
        trimmed_beams.sort(key=lambda x: x.score_lm / len(x.prediction), reverse=True)
    else:
        trimmed_beams.sort(key=lambda x: x.score_lm, reverse=True)
    return trimmed_beams

In [18]:
# r = sess.run([decoded, features['targets'], encoded, features['inputs_length'][:,0], initial_states])
# l = r[3] // conformer_model.conv_subsampling.time_reduction_factor

In [19]:
# for no, row in enumerate(r[0]):
#     d = malaya_speech.subword.decode(subwords, row[row > 0])
#     t = malaya_speech.subword.decode(subwords, r[1][no])
#     print(no, t)
#     print(no, d)

In [20]:
# language_model = LanguageModel(kenlm_model, alpha = 0.05, beta = 1.0)

# o = transducer_lm(
#     enc = r[2][0],
#     total = l[0],
#     initial_states = r[4],
#     encoded_placeholder = encoded_placeholder,
#     predicted_placeholder = predicted_placeholder,
#     states_placeholder = states_placeholder,
#     ytu = ytu, 
#     new_states = new_states,
#     sess = sess,
#     beam_width = 5,
# )
# # malaya_speech.subword.decode(subwords, o[0])
# [(malaya_speech.subword.decode(subwords, i.prediction), i.score_lm) for i in o]

In [21]:
wer, cer = [], []
wer_lm, cer_lm = [], []
index = 0
while True:
    try:
        r = sess.run([decoded, features['targets'], encoded, features['inputs_length'][:,0], initial_states])
        l = r[3] // conformer_model.conv_subsampling.time_reduction_factor
        for no, row in enumerate(r[0]):
            d = malaya_speech.subword.decode(subwords, row[row > 0])
            t = malaya_speech.subword.decode(subwords, r[1][no])
            
            wer.append(malaya_speech.metrics.calculate_wer(t, d))
            cer.append(malaya_speech.metrics.calculate_cer(t, d))
            
            o = transducer_lm(
                enc = r[2][no],
                total = l[no],
                initial_states = r[4],
                encoded_placeholder = encoded_placeholder,
                predicted_placeholder = predicted_placeholder,
                states_placeholder = states_placeholder,
                ytu = ytu,
                new_states = new_states,
                sess = sess,
                beam_width = 3,
            )[0].prediction
            d_lm = malaya_speech.subword.decode(subwords, o)

            wer_lm.append(malaya_speech.metrics.calculate_wer(t, d_lm))
            cer_lm.append(malaya_speech.metrics.calculate_cer(t, d_lm))
        index += 1
        print(index)
    except Exception as e:
        break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260


In [22]:
d, t, d_lm

('saja tempat banding menaragis buatannya itu pilih saja yang isinya',
 'saji tempat bangdik menaruh tumis tahu buatannya itu piring saji yang isinya',
 'saja tempat banding menaramis buatannya itu pilih saja yang isinya')

In [23]:
np.mean(wer), np.mean(cer), np.mean(wer_lm), np.mean(cer_lm)
# (0.24665871358025013, 0.11013793851962718)

(0.21284191174535216,
 0.08140905442040013,
 0.19968285285321455,
 0.07700373888679218)